In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from PIL import Image

# === PATH DASAR ===
base_path = "/content/drive/MyDrive/DatasetPDL/Datasets"
csv_path = os.path.join(base_path, "_annotations.csv")  # ganti jika nama file beda
img_dir = os.path.join(base_path, "images")
label_dir = os.path.join(base_path, "labels")
os.makedirs(label_dir, exist_ok=True)

# === BACA CSV TANPA HEADER ===
# Tambahkan header kolom secara manual
df = pd.read_csv(csv_path, header=None, names=["filename", "xmin", "ymin", "xmax", "ymax"])
df['class'] = 0  # hanya ada 1 kelas: plastik

# === KONVERSI KE FORMAT YOLO ===
for filename in df['filename'].unique():
    img_path = os.path.join(img_dir, filename)
    label_path = os.path.join(label_dir, filename.replace('.jpg', '.txt').replace('.JPG', '.txt'))

    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue

    try:
        img = Image.open(img_path)
        width, height = img.size
    except:
        print(f"Error reading image: {img_path}")
        continue

    rows = df[df['filename'] == filename]

    with open(label_path, "w") as f:
        for _, row in rows.iterrows():
            xmin, ymin, xmax, ymax = row[['xmin','ymin','xmax','ymax']]
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

In [3]:
from sklearn.model_selection import train_test_split
import shutil

# Buat folder split YOLOv8
yolo_path = "/content/yolo_dataset"
for split in ['train', 'val']:
    os.makedirs(f"{yolo_path}/images/{split}", exist_ok=True)
    os.makedirs(f"{yolo_path}/labels/{split}", exist_ok=True)

# Split data
filenames = df['filename'].unique()
train_files, val_files = train_test_split(filenames, test_size=0.2, random_state=42)

# Copy data
for file_list, split in zip([train_files, val_files], ['train', 'val']):
    for fname in file_list:
        shutil.copy(os.path.join(img_dir, fname), f"{yolo_path}/images/{split}/{fname}")
        txt_name = fname.replace('.jpg', '.txt')
        shutil.copy(os.path.join(label_dir, txt_name), f"{yolo_path}/labels/{split}/{txt_name}")


In [4]:
with open(f"{yolo_path}/data.yaml", "w") as f:
    f.write("train: /content/yolo_dataset/images/train\n")
    f.write("val: /content/yolo_dataset/images/val\n")
    f.write("nc: 1\n")
    f.write("names: ['plastic']\n")


In [5]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # versi ringan
model.train(data=f"{yolo_path}/data.yaml", epochs=50, imgsz=640)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

100%|██████████| 6.25M/6.25M [00:00<00:00, 177MB/s]


Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

100%|██████████| 755k/755k [00:00<00:00, 120MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      5.22G      1.397     0.8856      1.162        233        640: 100%|██████████| 60/60 [00:16<00:00,  3.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.16it/s]


                   all        238       2559      0.797      0.645      0.744      0.421

50 epochs completed in 0.287 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.03it/s]


                   all        238       2559      0.768       0.66      0.744      0.421
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fab1fdb12d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
# Evaluasi model
model.val()

# Contoh prediksi
model.predict(source="/content/yolo_dataset/images/val", save=True)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1122.2±515.5 MB/s, size: 33.0 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:05<00:00,  2.85it/s]


                   all        238       2559      0.771      0.658      0.744      0.421
Speed: 0.5ms preprocess, 7.1ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/train2

image 1/238 /content/yolo_dataset/images/val/plastic-10-_jpg.rf.1699574eff146fbd41f739de122652b7.jpg: 320x640 6 plastics, 206.9ms
image 2/238 /content/yolo_dataset/images/val/plastic-10-_jpg.rf.658cbd841efe9e2a7cb59c63aec94375.jpg: 320x640 5 plastics, 50.5ms
image 3/238 /content/yolo_dataset/images/val/plastic-100-_jpg.rf.ae0e4e178a0a8f9b1af20b2a13b23949.jpg: 448x640 17 plastics, 60.0ms
image 4/238 /content/yolo_dataset/images/val/plastic-101-_jpg.rf.811b01f69d12419be472853ff2084e14.jpg: 480x640 7 plastics, 70.4ms
image 5/238 /content/yolo_dataset/images/val/plastic-11-_jpg.rf.2572f37d983f8c4dc1659809cb2fdeb6.jpg: 320x640 1 plastic, 12.7ms
image 6/238 /content/yolo_dataset/images/val/plastic-11-_jpg.rf.b0c97a3167d0885c850d459959ad2121.jpg: 320x640 1 plastic, 12.9ms
image 7/238 /con

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'plastic'}
 obb: None
 orig_img: array([[[23, 23, 23],
         [23, 23, 23],
         [23, 23, 23],
         ...,
         [23, 23, 23],
         [23, 23, 23],
         [23, 23, 23]],
 
        [[23, 23, 23],
         [23, 23, 23],
         [23, 23, 23],
         ...,
         [23, 23, 23],
         [23, 23, 23],
         [23, 23, 23]],
 
        [[23, 23, 23],
         [23, 23, 23],
         [23, 23, 23],
         ...,
         [23, 23, 23],
         [23, 23, 23],
         [23, 23, 23]],
 
        ...,
 
        [[23, 23, 23],
         [23, 23, 23],
         [23, 23, 23],
         ...,
         [23, 23, 23],
         [23, 23, 23],
         [23, 23, 23]],
 
        [[23, 23, 23],
         [23, 23, 23],
         [23, 23, 23],
         ...,
         [23, 23, 23],
         [23, 23, 23],
         [23, 23, 23]],
 
        [[23, 23, 23],
   

In [8]:
!zip -r /content/yolo_output.zip /content/runs /content/yolo_dataset

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 35%)
  adding: content/runs/detect/train/train_batch2400.jpg (deflated 13%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 5%)
  adding: content/runs/detect/train/train_batch2401.jpg (deflated 12%)
  adding: content/runs/detect/train/train_batch1.jpg (deflated 4%)
  adding: content/runs/detect/train/train_batch0.jpg (deflated 7%)
  adding: content/runs/detect/train/F1_curve.png (deflated 16%)
  adding: content/runs/detect/train/train_batch2402.jpg (deflated 15%)
  adding: content/runs/detect/train/args.yaml (deflated 53%)
  adding: content/runs/detect/train/val_batch1_pred.jpg (deflated 10%)
  adding: content/runs/detect/train/labels.jpg (deflated 33%)
  adding: content/runs/detect/train/val_batch0_pred.jpg (deflated 10%)
  adding: content/runs/detect/train/val_batch2_labels.jpg

In [9]:
from google.colab import files
files.download("/content/yolo_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>